# Processing SARG outputfile for ARG ranking

In [1]:
import pandas as pd

## Precessing dmnd file

In [16]:
#coveragy cal
input_sarg="/lomi_home/gaoyang/software/CompRanking/test/CompRanking/CompRanking_intermediate/AMR/ARGranking/ERR1191817.contigs_5M_contigs_SARG_Protein_diamond.txt"
input_sarg="/lomi_home/gaoyang/software/CompRanking/benchmarking/MGE_benchmarking/plasmid_MOB/non_mob_plas/nonmobplas/CompRanking_intermediate/AMR/ARGranking/nonmob_plas_5M_contigs_SARG_Protein_diamond.txt"
df_sarg=pd.read_csv(input_sarg, sep="\t",header=None)
df_sarg.columns = ['id', 'sub_id', 'identity', 'alignLen', 'mismat', 'gapOpens', 'qStart', 'qEnd', 'sStart', 'sEnd', 'eval', 'bit']

#open df_sarg
df_SARG_len=pd.read_csv("/lomi_home/gaoyang/software/CompRanking/databases/SARG/SARG.db.fasta.length", sep="\t", header=None )
df_SARG_len=df_SARG_len[[1,2]]

#merge len and df_sarg
df_sarg_len=pd.merge(df_sarg,df_SARG_len,left_on="sub_id",right_on=1,how="left")
df_sarg_len["coverage"]="-"

#cover cal
for index, name in df_sarg_len.iterrows():
    df_sarg_len["coverage"][index]=df_sarg_len["alignLen"][index]/df_sarg_len[2][index]

# filter out contigs identity under 80
df_sarg_iden95 = df_sarg_len[df_sarg_len.identity > 80]
# filter out contigs alignment length under 0.75
df_sarg_alen = df_sarg_iden95[df_sarg_iden95.coverage>0.75]
df_sarg_alen
df_sarg_tmp=df_sarg_alen.iloc[:,[0,1]]
df_sarg_tmp.columns=["Contig_ID", "query"]
df_sarg_tmp

/tmp/ipykernel_303897/45944870.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sarg_len["coverage"][index]=df_sarg_len["alignLen"][index]/df_sarg_len[2][index]


,Contig_ID,query


In [17]:
input_sarg_structure="/lomi_home/gaoyang/software/CompRanking/databases/SARG/SARG.structure.txt"
df_sarg_structure=pd.read_csv(input_sarg_structure, sep="\t", header=0)
df_sarg_structure

,Reference,Genotype,Phenotype
0,0910185A,aminoglycoside__aph(3)-IV,aminoglycoside
1,1112175A,macrolide-lincosamide-streptogramin__ermE,macrolide-lincosamide-streptogramin
2,1405331A,aminoglycoside__streptomycin resistance protein,aminoglycoside
3,1411197A,beta-lactam__class D beta-lactamase,beta-lactam
4,1616230A,beta-lactam__class A beta-lactamase,beta-lactam
...,...,...,...
4045,ZP_04668102,tetracycline__tetO,tetracycline
4046,ZP_04677690,multidrug__norA,multidrug
4047,ZP_04677702,bacitracin__bacA,bacitracin
4048,ZP_04679156,multidrug__mepA,multidrug


In [18]:
df_sarg_type=pd.merge(df_sarg_tmp,df_sarg_structure,left_on="query",right_on="Reference",how="left")
df_sarg_type1=df_sarg_type.drop(['Reference'], axis=1, inplace=False)
df_sarg_type1

,Contig_ID,query,Genotype,Phenotype


In [19]:
# df_argrank1.columns=["Contig_ID","query","class","Phenotype","SARG_Rank"]
if len(df_sarg_type1["Contig_ID"]) == 0:
    print("true")
    df_sarg_type1["Phenotype"]=[]
    df_sarg_type1["SARG_Rank"]=[]
    df_sarg_type1.columns=["Contig_ID","query","class","Phenotype","SARG_Rank"]
else:
    pass

df_sarg_type1

true


,Contig_ID,query,class,Phenotype,SARG_Rank


In [5]:
#standardizing format
df_sarg_type1["class"]=df_sarg_type1["Genotype"].str.split("__",expand=True)[1]
df_sarg_type2=df_sarg_type1.drop(['Genotype'], axis=1, inplace=False)
df_sarg_type2=df_sarg_type2[["Contig_ID","query","class","Phenotype"]]
df_sarg_type2

KeyError: 1

## Processing arg ranking index

In [31]:
#open arg rank index file
input_argrank="/lomi_home/gaoyang/software/CompRanking/databases/SARG/ARG_rank.txt"
df_argrank=pd.read_csv(input_argrank, sep="\t", header=0)
df_argrank

,ARG,Genotype,Phenotype,Enrichment,Mobility,ESKAPE_pathogen,Plasmid,Integron,Host_range,Human_associated_occ,Human_associated_abun,Enrichment_value,Rank
0,AAB20442,aminoglycoside__aac(3)-II,aminoglycoside,TRUE,TRUE,TRUE,TRUE,False,class,0.0470,1.00E-03,Human_only,I
1,YP_001966247,aminoglycoside__aac(3)-II,aminoglycoside,TRUE,TRUE,TRUE,TRUE,False,class,0.0840,4.40E-03,Human_only,I
2,NC_009838.1.5616796.p01,aminoglycoside__aac(3)-VI,aminoglycoside,TRUE,TRUE,TRUE,TRUE,False,family,0.0041,4.20E-03,322.544845,I
3,ABG77519,aminoglycoside__aac(6')-I,aminoglycoside,TRUE,TRUE,TRUE,TRUE,True,phylum,0.0690,4.10E-03,416.195466,I
4,AF231133.1.gene2.p01,aminoglycoside__aac(6')-I,aminoglycoside,TRUE,TRUE,TRUE,TRUE,False,phylum,0.0160,3.90E-04,Human_only,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4045,DQ018711.1.gene7.p01,vancomycin__vanY,vancomycin,None,None,None,None,False,NaN,0.4700,4.40E-03,21.69590399,Unassessed
4046,FJ349556.1.gene4.p01,vancomycin__vanY,vancomycin,None,None,None,None,False,NaN,0.4900,5.40E-03,7.498609207,Unassessed
4047,AF155139.2.gene5.p01,vancomycin__vanZ,vancomycin,None,None,None,None,False,NaN,0.1400,2.70E-03,1.699360876,Unassessed
4048,CAB61225,vancomycin__vanZ,vancomycin,None,None,None,None,False,NaN,0.0020,4.50E-05,0.001356539,Unassessed


In [32]:
df_arg1=df_argrank[["ARG","Rank"]]
df_arg1

,ARG,Rank
0,AAB20442,I
1,YP_001966247,I
2,NC_009838.1.5616796.p01,I
3,ABG77519,I
4,AF231133.1.gene2.p01,I
...,...,...
4045,DQ018711.1.gene7.p01,Unassessed
4046,FJ349556.1.gene4.p01,Unassessed
4047,AF155139.2.gene5.p01,Unassessed
4048,CAB61225,Unassessed


In [33]:
df_argrank2=pd.merge(df_sarg_type2,df_arg1,left_on="query",right_on="ARG",how="inner")
df_argrank2=df_argrank2.drop(["ARG"],axis=1,inplace=False)
df_argrank2.columns=["Contig_ID","query","class","Phenotype","SARG_Rank"]
df_argrank2

,Contig_ID,query,class,Phenotype,SARG_Rank
0,240_1,ABB89122,dfrA5,trimethoprim,I
1,240_2,AB104852.1.gene2.p01,IMP-1,beta-lactam,Unassessed
2,1157_2,ACP19239,vanZ,vancomycin,IV
3,1157_3,AAA65958,vanY,vancomycin,I
4,1157_4,ACC93634,vanX,vancomycin,IV
...,...,...,...,...,...
106,117583_3,AAF72359,vanS,vancomycin,III
107,117583_4,KF823969.1.gene1.p01,vanR,vancomycin,III
108,121135_4,YP_112402,aph(3)-I,aminoglycoside,IV
109,121356_1,CAG34229,ant(2)-I,aminoglycoside,I


In [34]:
df_argrank2.to_csv("./test_SARGrank_Protein90_Result_tmp.txt", sep="\t")